In [41]:
import numpy as np
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T
import time

In [42]:
@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def main(A: T.Buffer[128, "float32"],
             B: T.Buffer[128, "float32"],
             C: T.Buffer[128, "float32"]):
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        for i in range(128):
            with T.block("C"):
                vi = T.axis.spatial(128, i)
                C[vi] = A[vi] + B[vi]

In [43]:
def add(a, b, c):
    for i in range(128):
        c[i] = a[i] + b[i]

In [44]:
MyModule

#[version = "0.0.5"]
@main = primfn(A_handle: handle, B_handle: handle, C_handle: handle) -> ()
  attr = {"tir.noalias": True, "global_symbol": "main"}
  buffers = {A: Buffer(A_1: Pointer(global float32), float32, [128], []),
             B: Buffer(B_1: Pointer(global float32), float32, [128], []),
             C: Buffer(C_1: Pointer(global float32), float32, [128], [])}
  buffer_map = {A_handle: A, B_handle: B, C_handle: C} {
  block([], "root") {
    tir.reads([])
    tir.writes([])
    for (i: int32, 0, 128) {
      block([128], "C") as [vi] {
        bind(vi, i)
        tir.reads([A[vi], B[vi]])
        tir.writes([C[vi]])
        C[vi] = (A[vi] + B[vi])
    }
}

#[metadata]
{
  "root": 1, 
  "nodes": [
    {
      "type_key": ""
    }, 
    {
      "type_key": "Map", 
      "keys": [
        "IntImm"
      ], 
      "data": [2]
    }, 
    {
      "type_key": "Array", 
      "data": [3]
    }, 
    {
      "type_key": "IntImm", 
      "attrs": {
        "dtype": "bool", 
        "

In [45]:
sch = tvm.tir.Schedule(MyModule)

In [46]:
block_c = sch.get_block("C")

In [47]:
i, = sch.get_loops(block_c)

In [48]:
i0, i1, i2 = sch.split(i, factors=[None, 2, 4])

In [49]:
print(sch.mod.script())

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[128, "float32"], B: T.Buffer[128, "float32"], C: T.Buffer[128, "float32"]):
        # function attr dict
        T.func_attr({"tir.noalias": True, "global_symbol": "main"})
        # body
        # with T.block("root")
        for i_0, i_1, i_2 in T.grid(16, 2, 4):
            with T.block("C"):
                vi = T.axis.spatial(128, i_0 * 8 + i_1 * 4 + i_2)
                T.reads(A[vi], B[vi])
                T.writes(C[vi])
                C[vi] = A[vi] + B[vi]
    



In [50]:
sch.reorder(i2, i1)
print(sch.mod.script())

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[128, "float32"], B: T.Buffer[128, "float32"], C: T.Buffer[128, "float32"]):
        # function attr dict
        T.func_attr({"tir.noalias": True, "global_symbol": "main"})
        # body
        # with T.block("root")
        for i_0, i_2, i_1 in T.grid(16, 4, 2):
            with T.block("C"):
                vi = T.axis.spatial(128, i_0 * 8 + i_1 * 4 + i_2)
                T.reads(A[vi], B[vi])
                T.writes(C[vi])
                C[vi] = A[vi] + B[vi]
    



In [33]:
sch.parallel(i1)
print(sch.mod.script())

# from tvm.script import tir as T
@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[128, "float32"], B: T.Buffer[128, "float32"], C: T.Buffer[128, "float32"]):
        # function attr dict
        T.func_attr({"tir.noalias": True, "global_symbol": "main"})
        # body
        # with T.block("root")
        for i_0, i_2 in T.grid(8, 4):
            for i_1 in T.parallel(4):
                with T.block("C"):
                    vi = T.axis.spatial(128, i_0 * 16 + i_1 * 4 + i_2)
                    T.reads(A[vi], B[vi])
                    T.writes(C[vi])
                    C[vi] = A[vi] + B[vi]
    



In [51]:
rt_mod = tvm.build(sch.mod, "llvm", name="myadd")

In [52]:
funct = rt_mod["main"]
type(funct)

tvm.runtime.packed_func.PackedFunc

In [53]:
a = tvm.nd.array(np.arange(128).astype("float32"))
b = tvm.nd.array(np.ones(128).astype("float32"))
c = tvm.nd.empty([128], "float32")
type(a)


tvm.runtime.ndarray.NDArray

In [39]:
time1 = time.time()
funct(a, b, c)
time2 = time.time()
print(time2 - time1)
c

0.0036890506744384766


<tvm.nd.NDArray shape=(128,), cpu(0)>
array([  1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
        12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,
        23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,
        34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,
        45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,
        56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,  65.,  66.,
        67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,
        78.,  79.,  80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,
        89.,  90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.,
       100., 101., 102., 103., 104., 105., 106., 107., 108., 109., 110.,
       111., 112., 113., 114., 115., 116., 117., 118., 119., 120., 121.,
       122., 123., 124., 125., 126., 127., 128.], dtype=float32)

In [40]:
res = b.asnumpy() + a.asnumpy() - c.asnumpy()
res

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)